In [1]:
from api_token import token
import todoist as td
import pandas as pd
import datetime as dt

In [2]:
api = td.TodoistAPI(token)
# Sync to API to get latest data
api.sync();

In [3]:
# when the api query has no items (tasks) to report, it returns a dictionary with two empty keys e.g.
# {'items':[], 'projects':[]}
# Therefore I need to create a function to check if the keys in the dictionary have values

def has_values(dictionary):
    
    # For each key in dictionary, check if keys in dict have values
    values = [any(dictionary[key]) for key in dictionary]
    
    # Create a unique list of values
    output = list(set(values))
    
    return output[0]

In [4]:
# The API has a maximum of 200 items that can be returned, use while loop to get all items

completed_tasks_list = []
limit = 200
offset = 0
not_empty = True

while not_empty:
    # API call, retrieving between 0 and 200 items at a time
    completed_tasks = api.completed.get_all(limit=limit, offset=offset)
    not_empty = has_values(completed_tasks) # if false it will exit the loop
    completed_tasks_list.append(completed_tasks) # add items to list
    offset += limit # increase offset to get the next set of results
    

In [5]:
# Create a list of dfs for each result of items
completed_dfs = [pd.DataFrame(completed_tasks_list[i]['items']) for i in range(0,len(completed_tasks_list))]

In [6]:
# Combine dfs into one large df
df = pd.concat(completed_dfs)

In [7]:
# Check num of rows and columns
df.shape

(952, 7)

In [30]:
# Our df doesn't yet have project names, only project ids, we want to add a field with project names
td_projects_dict = {}
for i,data in enumerate(completed_tasks_list):
    for key in data['projects'].keys():
        td_projects_dict[int(key)] = data['projects'][key]['name']

In [25]:
# Map project names to df
df['project_name'] = df['project_id'].map(td_projects_dict)

In [28]:
# df.head()

In [16]:
# TODO Create statement that prints the following:
# You completed XXX tasks in 2020. That’s an average of…
# XX per day
# XX tasks per week
# XX tasks by month

In [17]:
# TODO first add a column that will get the year of completion